# 線形計画をPythonで解いてみる

[入門オペレーションズリサーチ](https://www.amazon.co.jp/dp/4486017447) の「第12章 仕事の効率を高める 線形計画」の例題をPythonで解いてみる。

参考文献： [PuLP による線型計画問題の解き方ことはじめ - Qiita](https://qiita.com/mzmttks/items/82ea3a51e4dbea8fbc17)

## 12.1 アイス増産計画 （p.151）

* 2種類のアイスの生産を計画
* 材料の牛乳は全部で8000cc
* 作業時間は延べ360分
* 儲けを最大にする増産計画を求めたい。

| | 牛乳 | 作業時間 | 儲け |
|:---|:---|:---|:---|
| エスプレッソアイス | 100cc | 7分 | 250円 |
| ラズベリーアイス | 150cc | 5分 | 300円 |

In [1]:
#!pip install pulp
import pulp

エスプレッソアイスの生産量を `x1` 、ラズベリーアイスの生産量を `x2` とする。

変数の定義時に最小値最大値を設定する必要があるが、最大値はここでは便宜上 `9999` と置く。（PuLPを使う場合、変数の定義の時点でそれが制約の一部を兼ねる模様）"


In [2]:
x1 = pulp.LpVariable('x1', 0, 9999, 'Integer') 
x2 = pulp.LpVariable('x2', 0, 9999, 'Integer') 


この時、儲けの合計 `(250 * x1 + 300 * x2)` を最大化したい。（目的）

In [3]:

problem = pulp.LpProblem('アイス増産計画', pulp.LpMaximize)

# 目的関数
problem += 250 * x1 + 300 * x2


次に制約条件を追加していく。

まず、牛乳の使用量の制約。牛乳は合計8000ccまでしか使えないので、各アイスにおける使用量の合計の上限を8000ccとする。


In [4]:
# 使用量の制約条件
problem += 100 * x1 + 150 * x2 <= 8000

次に生産時間の制約。増産に使える時間は延べ360分なので、各アイスにおける生産時間の合計の上限を360分とする。


In [5]:
# 生産時間の制約条件
problem += 7 * x1 + 5 * x2 <= 360

書籍で記載されている `x1` および `x2` が正の値 `(x1, x2 >= 0)` は変数定義時に最小値として設定してあるので、割愛する。

解く。

In [6]:
status = problem.solve()
print(pulp.LpStatus[status])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/80608731fa9244a6bf56fdefabe6fee6-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/80608731fa9244a6bf56fdefabe6fee6-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 18 RHS
At line 21 BOUNDS
At line 24 ENDATA
Problem MODEL has 2 rows, 2 columns and 4 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 17272.7 - 0.00 seconds
Cgl0004I processed model has 2 rows, 2 columns (2 integer (0 of which binary)) and 4 elements
Cutoff increment increased from 1e-05 to 49.9999
Cbc0012I Integer solution of -17050 found by DiveCoefficient after 0 iterations and 0 nodes (0.01 seconds)
Cbc0012I Integer solution of -17150 found by DiveCoefficient after 8 iterations and 0 nodes (0.01 seconds)
C

`Optimal` が出ていれば解けている。

問題を改めて見てみる。

In [7]:
print(problem)

アイス増産計画:
MAXIMIZE
250*x1 + 300*x2 + 0
SUBJECT TO
_C1: 100 x1 + 150 x2 <= 8000

_C2: 7 x1 + 5 x2 <= 360

VARIABLES
0 <= x1 <= 9999 Integer
0 <= x2 <= 9999 Integer



求めた値を見てみる。

まずは `x1` 、エスプレッソアイスの生産量。

In [8]:
x1.value()

23.0

次に `x2` 、ラズベリーアイスの生産量。

In [9]:
x2.value()

38.0

つまり、制約条件のもとで、エスプレッソアイスを 23、ラズベリーアイスを 38 生産した時に、儲けが最大化される。